# Statistical Modeling

In [1]:
import math

In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib
import matplotlib.pyplot as pp

In [4]:
%matplotlib inline

In [5]:
import pymc3 as pm

### Bayesian inference

A type of statistical inference

- We do not make ***estimates*** of population parameters
- Rather, we maintain ***probablity distributions*** for population distributions, which represent our quantitative belief about their value
- We start with probablity ***priors***, and we use ***observations*** to update them to probablity ***posteriors***

***Prior *** p(1) = p(2) = 0.5 --->

***Observation *** <i> After observation if we are more incline to to p(1) </i> ---> 

***Posterior *** p(1) = 1, p(2) = 0

#### Bayes Theorem

For more general distributions the update rule is called "Bayes Theorem"

Posterior  Likelihood+   Prior
p(θ|y) =      p(y|θ)     p(θ)   /   ∫p(y|θ)p(θ)d(θ)

+Likelihood: probablity of gettting data for given population parameters

Small note: ∫ == the integral symbol